In [1]:
# Libraries
import unicodedata
from xlrd import open_workbook
import os

In [2]:
def rem_uni(u):
    '''
    this function removes unicode and returns a string
    '''
    return unicodedata.normalize('NFKD',u).encode('ascii','ignore')

In [3]:
#setup
all = open_workbook('C:\Users\\cacerb1\Documents\EDO-Intern-Project\Brittany\Directors\DBPedia\TopTenAttributes.xlsx')
os.chdir('C:\\Users\cacerb1\Documents\EDO-Intern-Project\Brittany\Directors\DBPedia')

In [4]:
def make_nodes(s):
    '''
    Generatres Cypher code for nodes
    '''
    query='CREATE '
    for r in range (1,s.nrows):
        q='('+rem_uni(s.cell(r,0).value)+':'+rem_uni(s.cell(r,1).value)+' { '
        for c in range(2,s.ncols):
            if s.cell(r,c).value!= 'NA':
                q+= rem_uni(s.cell(0,c).value)+': "'+rem_uni(s.cell(r,c).value)+'", '
        q=q[0:-2]
        q+=' }),'
        query+=q
    return query[0:-1]

In [5]:
def make_rel(s):
    '''
    Generates Cypher Code for relationships'''
    a='CREATE'
    for r in range (1,s.nrows):
        q='('+rem_uni(s.cell(r,0).value)+')-[:'+rem_uni(s.cell(r,1).value)+']->('+rem_uni(s.cell(r,2).value)+'), '
        a+=q
    return a[0:-2]

In [6]:
#Make the Cypher
entry= make_nodes(all.sheets()[0])
rel= make_rel(all.sheets()[1])

In [7]:
with open('dquery.txt',"wt") as out:
    out.write(entry+rel)

In [8]:
from neo4j.v1 import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "Anc1r3a."))

def create(tx,s):
    tx.run(s)
    
with driver.session() as session:
    session.write_transaction(create,'MATCH (n) DETACH DELETE n')
    session.write_transaction(create,entry+rel)